In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName('lr_example').getOrCreate()

In [3]:
from pyspark.ml.regression import LinearRegression

In [4]:
data = spark.read.csv('course-material/Spark_for_Machine_Learning/Linear_Regression/Ecommerce_Customers.csv', inferSchema=True, header=True)

In [5]:
data.printSchema()

root
 |-- Email: string (nullable = true)
 |-- Address: string (nullable = true)
 |-- Avatar: string (nullable = true)
 |-- Avg Session Length: double (nullable = true)
 |-- Time on App: double (nullable = true)
 |-- Time on Website: double (nullable = true)
 |-- Length of Membership: double (nullable = true)
 |-- Yearly Amount Spent: double (nullable = true)



In [6]:
for i in data.head(2)[1]:
    print(i)

hduke@hotmail.com
4547 Archer CommonDiazchester, CA 06566-8576
DarkGreen
31.92627202636016
11.109460728682564
37.268958868297744
2.66403418213262
392.2049334443264


In [7]:
from pyspark.ml.linalg import Vector

In [8]:
from pyspark.ml.feature import VectorAssembler

In [9]:
data.columns

['Email',
 'Address',
 'Avatar',
 'Avg Session Length',
 'Time on App',
 'Time on Website',
 'Length of Membership',
 'Yearly Amount Spent']

In [10]:
assembler = VectorAssembler(inputCols=[
    'Avg Session Length',
 'Time on App',
 'Time on Website',
 'Length of Membership'], outputCol='features')

In [11]:
output = assembler.transform(data)

In [12]:
output.select(output['features']).show()

+--------------------+
|            features|
+--------------------+
|[34.4972677251122...|
|[31.9262720263601...|
|[33.0009147556426...|
|[34.3055566297555...|
|[33.3306725236463...|
|[33.8710378793419...|
|[32.0215955013870...|
|[32.7391429383803...|
|[33.9877728956856...|
|[31.9365486184489...|
|[33.9925727749537...|
|[33.8793608248049...|
|[29.5324289670579...|
|[33.1903340437226...|
|[32.3879758531538...|
|[30.7377203726281...|
|[32.1253868972878...|
|[32.3388993230671...|
|[32.1878120459321...|
|[32.6178560628234...|
+--------------------+
only showing top 20 rows



In [13]:
output.head(1)

[Row(Email='mstephenson@fernandez.com', Address='835 Frank TunnelWrightmouth, MI 82180-9605', Avatar='Violet', Avg Session Length=34.49726772511229, Time on App=12.65565114916675, Time on Website=39.57766801952616, Length of Membership=4.0826206329529615, Yearly Amount Spent=587.9510539684005, features=DenseVector([34.4973, 12.6557, 39.5777, 4.0826]))]

In [14]:
final_data = output.select('features', 'Yearly Amount Spent')

In [15]:
final_data.show()

+--------------------+-------------------+
|            features|Yearly Amount Spent|
+--------------------+-------------------+
|[34.4972677251122...|  587.9510539684005|
|[31.9262720263601...|  392.2049334443264|
|[33.0009147556426...| 487.54750486747207|
|[34.3055566297555...|  581.8523440352177|
|[33.3306725236463...|  599.4060920457634|
|[33.8710378793419...|   637.102447915074|
|[32.0215955013870...|  521.5721747578274|
|[32.7391429383803...|  549.9041461052942|
|[33.9877728956856...|  570.2004089636196|
|[31.9365486184489...|  427.1993848953282|
|[33.9925727749537...|  492.6060127179966|
|[33.8793608248049...|  522.3374046069357|
|[29.5324289670579...|  408.6403510726275|
|[33.1903340437226...|  573.4158673313865|
|[32.3879758531538...|  470.4527333009554|
|[30.7377203726281...|  461.7807421962299|
|[32.1253868972878...| 457.84769594494855|
|[32.3388993230671...| 407.70454754954415|
|[32.1878120459321...|  452.3156754800354|
|[32.6178560628234...|   605.061038804892|
+----------

In [16]:
train_data, test_data = final_data.randomSplit([0.7, 0.3])

In [17]:
train_data.describe().show()

+-------+-------------------+
|summary|Yearly Amount Spent|
+-------+-------------------+
|  count|                352|
|   mean|  499.7858988638776|
| stddev|  78.40006194841108|
|    min|  282.4712457199145|
|    max|  765.5184619388373|
+-------+-------------------+



In [18]:
test_data.describe().show()

+-------+-------------------+
|summary|Yearly Amount Spent|
+-------+-------------------+
|  count|                148|
|   mean| 498.19177519736746|
| stddev|  81.70861228470005|
|    min| 256.67058229005585|
|    max|  689.7876041747194|
+-------+-------------------+



In [19]:
lr = LinearRegression(labelCol='Yearly Amount Spent')

In [20]:
lr_model = lr.fit(train_data)

In [21]:
test_results = lr_model.evaluate(test_data)

In [22]:
test_results.residuals.show()

+-------------------+
|          residuals|
+-------------------+
|-0.8843450825359582|
|-16.209929378559195|
|-6.3783938766580945|
|  3.047603911483179|
|  4.013141514144422|
| -4.821218643016266|
| 17.820716586604533|
| -2.386781869980666|
| -9.367441377218427|
|-1.5874864358169134|
| -6.654109037991645|
| -14.05514476096073|
|-18.368771722180554|
| -5.764704161372606|
|  5.439260408356176|
|  16.50987091461775|
|    2.2989828100458|
|  4.306503457076587|
|-0.3353076855751169|
|-3.5694533597886675|
+-------------------+
only showing top 20 rows



In [23]:
test_results.rootMeanSquaredError

9.616277723362478

In [24]:
test_results.r2

0.9860548655313605

In [25]:
final_data.describe().show()

+-------+-------------------+
|summary|Yearly Amount Spent|
+-------+-------------------+
|  count|                500|
|   mean|  499.3140382585909|
| stddev|   79.3147815497068|
|    min| 256.67058229005585|
|    max|  765.5184619388373|
+-------+-------------------+



In [26]:
unlabeled_data = test_data.select('features')

In [27]:
unlabeled_data.show()

+--------------------+
|            features|
+--------------------+
|[30.5743636841713...|
|[30.8162006488763...|
|[31.0613251567161...|
|[31.3091926408918...|
|[31.3584771924370...|
|[31.5171218025062...|
|[31.6098395733896...|
|[31.8186165667690...|
|[31.8279790554652...|
|[31.8627411090001...|
|[31.8745516945853...|
|[31.9365486184489...|
|[31.9563005605233...|
|[31.9673209478824...|
|[31.9764800614612...|
|[32.0180740106320...|
|[32.0444861274404...|
|[32.0542618511847...|
|[32.0705462209254...|
|[32.0775900443291...|
+--------------------+
only showing top 20 rows



In [28]:
predictions = lr_model.transform(unlabeled_data)

In [29]:
predictions.show()

+--------------------+------------------+
|            features|        prediction|
+--------------------+------------------+
|[30.5743636841713...| 442.9487588406016|
|[30.8162006488763...| 282.2962703270282|
|[31.0613251567161...| 493.9338519345597|
|[31.3091926408918...|429.67311392845045|
|[31.3584771924370...|  491.162808935331|
|[31.5171218025062...|  280.739639293402|
|[31.6098395733896...| 426.7248330645036|
|[31.8186165667690...| 448.8054552401163|
|[31.8279790554652...|449.37018892415995|
|[31.8627411090001...| 557.8856276098636|
|[31.8745516945853...|398.93935328425914|
|[31.9365486184489...| 441.2545296562889|
|[31.9563005605233...| 565.4947034693794|
|[31.9673209478824...|451.51454540102486|
|[31.9764800614612...|  325.155185625744|
|[32.0180740106320...|341.27323983069755|
|[32.0444861274404...| 445.9308463765042|
|[32.0542618511847...| 557.5681542119064|
|[32.0705462209254...| 533.0870952674115|
|[32.0775900443291...| 404.6025885788947|
+--------------------+------------